In [1]:
import h5py
import os.path
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gd
from utils.paths import SetupPaths
import matplotlib.patches as  mpatches
from matplotlib.ticker import FormatStrFormatter
from utils.get_summary_data import compile_summary
from utils.vectorCorrection import vectorCorrection as vector


paths = SetupPaths()

ModuleNotFoundError: No module named 'utils.paths'

In [20]:
plt.show();
plt.rcParams.update({'font.size':20,"xtick.direction":"in","ytick.direction":"in", 
                     "xtick.top":True, "ytick.right":True,"text.usetex":False,
                     "xtick.labelsize":18,"ytick.labelsize":18})


In [21]:
# defining equivalent redshifts
zs = {"z":np.array([0,1,2,3,4]), 
#       "zill":np.array([135,85,68,60,56]), 
      "ztng":np.array([99,50,33,25,21])}

# defining color palette for plotting
palette = {"Illustris dark": "#009292", "Illustris hydro": "#B6DAFF",
           "TNG dark": "#930200", "TNG hydro": "#FFB5DC",
           "dwarf":"olive","massive":"salmon", "difference":"#2C1D11", "difference2":"#464646"}

alphas = {"maj": 0.7, "min": 0.3}


# Initialization

In [22]:
# make functions to get data at the requested snapshot
def get_primmask(primstells, size):
    if size == "dwarf":
        mask = (primstells > 0.01) & (primstells < 0.5)
    elif size == "massive":
        mask = (primstells > 0.5) & (primstells < 10)
    return mask

def get_groupmask(groupmass, size):
    if size == "dwarf":
        mask = (groupmass > 8) & (groupmass < 50)
    elif size == "massive":
        mask = (groupmass > 100) & (groupmass < 650)
    return mask

In [23]:
class EmptyFile(Exception): pass
class SkipRedshift(Exception): pass


# Primary + Pair count

In [6]:
def get_counts( size, reals, errorprint=False, redshiftcutoff=True):    
    snapshots = np.arange(0,100,1)
    snapshots = np.delete(snapshots, np.where(snapshots==48)[0])
    redcutoff = 4.2
        
    redshifts = []
    medone, medtwo, medtot, medmaj, medmin, medpair, medmajfrac, medminfrac, medtotfrac = [], [], [], [], [], [], [], [], []
    quartone, quarttwo, quarttot, quartmaj, quartmin, quartpair, quartmajfrac, quartminfrac, quarttotfrac = [], [], [], [], [], [], [], [], []

    for snap in snapshots:  
        singleprims, doubleprims, totalprims = [], [], []
        majorpairs, minorpairs, totalpairs = [], [], []
        majfrac, minfrac, totfrac = [], [], []
        
        try:
            pair_path = f"TNG_{snap}_{reals}.hdf5"
            pair_data = h5py.File(f"{paths.path_pairs}{pair_path}", "r")
            
            if np.size(pair_data) == 0:
                raise EmptyFile
                
            redshift = pair_data['Header'].attrs['Redshift']
            
            if redshiftcutoff & ( redshift > redcutoff) :
                raise SkipRedshift
                
            if (len(pair_data['pairs']["hydro"]['Group ID']) == 0):    
                raise EmptyFile
                
            unpair = pair_data["unpaired"]["hydro"]
            unpairStells = np.array(unpair["Sub1 Stellar Mass"])
            unpairGroups = np.array(unpair["Group Mass"])
            unpairReals = np.array(unpair['Realization'])
            
            pair = pair_data["pairs"]["hydro"]
            priStell = np.array(pair["Sub1 Stellar Mass"])
            secStell = np.array(pair["Sub2 Stellar Mass"])
            pairGroups = np.array(pair["Group Mass"])
            pairReals = np.array(pair["Realization"])
            seps = np.array(pair["Separation"]) 
            
            # subset masks for unpaired
            unpair_pri = get_primmask(unpairStells, size)
            unpair_group = get_groupmask(unpairGroups, size)                
            
            pair_pri = get_primmask(priStell, size)
            pair_group = get_groupmask(pairGroups, size)

            majors = (secStell/priStell > 1/4)
            minors = (secStell/priStell > 1/10) & (secStell/priStell < 1/4)
            allpairs = (majors + minors)
            pair_lowsep = (seps > 10)

            # defining combined masks 
            unpair_mask = unpair_pri & unpair_group
            primary_mask = pair_pri & pair_group
            pair_mask = pair_pri & pair_group & pair_lowsep & allpairs
                                                           
            for real in np.unique(unpairReals):                  
                # make realization masks
                unpair_real = unpairReals == real
                pair_real = pairReals == real

                # make count values for single realization
                numone = np.count_nonzero(unpair_mask & unpair_real)
                numtwo = np.count_nonzero(primary_mask & pair_real)
                numtot = numone + numtwo
                nummaj = np.count_nonzero(pair_mask & pair_real & majors)
                nummin = np.count_nonzero(pair_mask & pair_real & minors)
                numpair = np.count_nonzero(pair_mask & pair_real)
                
                if numtot == 0:
                    continue
                
                # collect count vals for all reals
                singleprims.append(numone)
                doubleprims.append(numtwo)
                totalprims.append(numtot)
                majorpairs.append(nummaj)
                minorpairs.append(nummin)
                totalpairs.append(numpair)
                majfrac.append(nummaj / numtot)
                minfrac.append(nummin / numtot)
                totfrac.append(numpair / numtot)

            # create arrays of medians and quartiles~ 
            lower, upper = 0.5, 99.5                
            redshifts.append( redshift )
            medone.append(np.median( singleprims ))
            medtwo.append(np.median( doubleprims ))
            medtot.append(np.median( totalprims ))
            medmaj.append(np.median( majorpairs ))
            medmin.append(np.median( minorpairs ))
            medpair.append(np.median( totalpairs ))
            medmajfrac.append(np.median( majfrac ))
            medminfrac.append(np.median( minfrac ))
            medtotfrac.append(np.median( totfrac ))   
            quartone.append( np.percentile( singleprims, [lower, upper]))
            quarttwo.append( np.percentile( doubleprims, [lower, upper]))
            quarttot.append( np.percentile( totalprims, [lower, upper]))
            quartmaj.append( np.percentile( majorpairs, [lower, upper]))
            quartmin.append( np.percentile( minorpairs, [lower, upper]))
            quartpair.append( np.percentile( totalpairs, [lower, upper]))
            quartmajfrac.append( np.percentile( majfrac, [lower, upper]))
            quartminfrac.append( np.percentile( minfrac, [lower, upper]))
            quarttotfrac.append( np.percentile( totfrac, [lower, upper]))
                       
        except KeyError:
            if errorprint: print(f'skipping {snap} for KeyError. Please debug')
            continue
            
        except EmptyFile:
            if errorprint: print(f"skipping {snap}, empty file")
            continue
            
        except SkipRedshift:
            if errorprint: print(f"skipping {snap}, redshift out of range")
                
    count_dictionary = {
            "z": np.array(redshifts),
            "Median Isolated Primaries": np.array(medone),
            "Median Noniso Primaries": np.array(medtwo),
            "Median Total Primaries": np.array(medtot),
            "Median Major Pairs": np.array(medmaj),
            "Median Minor Pairs": np.array(medmin),
            "Median All Pairs": np.array(medpair),
            "Median Major Fraction": np.array(medmajfrac),
            "Median Minor Fraction": np.array(medminfrac),
            "Median Total Fraction": np.array(medtotfrac),
            "Quarts Isolated Primaries": np.array(quartone),
            "Quarts Noniso Primaries": np.array(quarttwo),
            "Quarts Total Primaries": np.array(quarttot),
            "Quarts Major Pairs": np.array(quartmaj),
            "Quarts Minor Pairs": np.array(quartmin),
            "Quarts All Pairs": np.array(quartpair),
            "Quarts Major Fraction": np.array(quartmajfrac),
            "Quarts Minor Fraction": np.array(quartminfrac),
            "Quarts Total Fraction": np.array(quarttotfrac)}
            
    return count_dictionary

In [7]:
def make_countdata(reals):
    #check if files exists
    filepath = f"{paths.path_plotdata}counts.hdf5"
    if not os.path.isfile(filepath):
        print("file does not exist...")
        print("creating file")
        f = h5py.File(f"{paths.path_plotdata}counts.hdf5", 'w')
        print("file created successfully. adding header...")
        header_dict = {"1000 Reals - Quartile Range":"0.5-99.5%",
                       "Simulation":"TNG100-1 (Hydro)"}


        dset = f.create_group('/Header')
        for key in header_dict.keys():
            dset.attrs[key] = header_dict[key]
            
        print("header added successfully")
    else:
        print("file exists...")
        f = h5py.File(f"{paths.path_plotdata}counts.hdf5", 'r+')
        
        
    print("checking to see if data exists for this number of realizations")
    
    if f.get(f"{reals} Realizations") is not None:
        print("data already exists!")
        f.close()
              
    else:
        print("data does not exist...")
        print("creating data tables...")
        dwarfs = get_counts("dwarf", reals)
        print("finished creating dwarf tables")
        massives = get_counts("massive", reals)
        print("finished creating massive tables")
              
        print("creating hdf5 structure")
        for size_data, size_name in zip([dwarfs,massives],["dwarf","massive"]):
            for key, val in size_data.items():
                val = np.array(val)
                dset = f.create_dataset(f'/{reals} Realizations/{size_name}/{key}', 
                                        shape=val.shape,
                                        dtype=val.dtype)
                dset[:] = val
                
        f.close()
        print("data saved~")

In [8]:
make_countdata(10)
make_countdata(100)
make_countdata(1000)

file exists...
checking to see if data exists for this number of realizations
data already exists!
file exists...
checking to see if data exists for this number of realizations
data already exists!
file exists...
checking to see if data exists for this number of realizations
data already exists!


# Mass ratio distribuition

In [15]:
def get_smr(size, z, reals):
    zloc = np.where( zs['z'] == z)[0]
    sim = "TNG"
    snapshot = zs['ztng'][zloc][0] 

    pair_path = f"{sim}_{snapshot}_{reals}.hdf5"
    pair_data = h5py.File(f"{paths.path_pairs}{pair_path}", "r")
    
    pairs = pair_data["pairs"]["hydro"]

    pri_stell = np.array(pairs["Sub1 Stellar Mass"])
    sec_stell = np.array(pairs["Sub2 Stellar Mass"])
    seps = np.array(pairs["Separation"]) 

    # masks            
    pair_pri = get_primmask(pri_stell, size)
    pair_group = get_groupmask(np.array(pairs["Group Mass"]), size)
    pair_sepcut = seps > 10
    
    majors = (sec_stell/pri_stell > 1/4)
    minors = (sec_stell/pri_stell > 1/10) & (sec_stell/pri_stell < 1/4)

    pair_mask = pair_pri & pair_group & pair_sepcut

    major_mask = pair_mask & majors
    minor_mask = pair_mask & minors

    majors = np.array(pairs["Stellar Mass Ratio"])[major_mask]
    minors = np.array(pairs["Stellar Mass Ratio"])[minor_mask]

    key_dict = {"major":majors, "minor":minors}
    

    return key_dict    

In [16]:
def make_smrdata(reals):
    #check if files exists
    filepath = f"{paths.path_plotdata}smr.hdf5"
    if not os.path.isfile(filepath):
        print("file does not exist...")
        print("creating file")
        f = h5py.File(filepath, 'w')
        print("file created successfully. adding header...")
        header_dict = {"Simulation":"TNG100-1 (Hydro)"}


        dset = f.create_group('/Header')
        for key in header_dict.keys():
            dset.attrs[key] = header_dict[key]
            
        print("header added successfully")
    else:
        print("file exists...")
        f = h5py.File(filepath, 'r+')
        
        
    print("checking to see if data exists for this number of realizations")
    
    if f.get(f"{reals} Realizations") is not None:
        print("data already exists!")
        f.close()
              
    else:
        print("data does not exist...")
        print("creating data tables...")
        
        for z in zs["z"]:
            dwarf = get_smr("dwarf", z, reals)
            massive = get_smr("massive", z, reals)
            print(f"finished z={z}")
              
            for size_data, size_name in zip([dwarf,massive],["dwarf","massive"]):
                for key, val in size_data.items():
                    val = np.array(val)
                    dset = f.create_dataset(f'/{reals} Realizations/z={z}/{size_name}/{key}', 
                                            shape=val.shape,
                                            dtype=val.dtype)
                    dset[:] = val
        print("data saved")        
        f.close()  

In [17]:
make_smrdata(10)
make_smrdata(100)
make_smrdata(1000)

file does not exist...
creating file
file created successfully. adding header...
header added successfully
checking to see if data exists for this number of realizations
data does not exist...
creating data tables...
finished z=0
finished z=1
finished z=2
finished z=3
finished z=4
data saved
file exists...
checking to see if data exists for this number of realizations
data does not exist...
creating data tables...
finished z=0
finished z=1
finished z=2
finished z=3
finished z=4
data saved
file exists...
checking to see if data exists for this number of realizations
data does not exist...
creating data tables...
finished z=0
finished z=1
finished z=2
finished z=3
finished z=4
data saved


# Pair fraction

In [18]:
def get_pairfrac(reals, errorprint=False, redshiftcutoff=True):    
    snapshots = np.arange(0,100,1)
    snapshots = np.delete(snapshots, np.where(snapshots==48)[0])
    redcutoff = 4.2
        
    redshifts = []
    medfrac_majdw, medfrac_mindw, medfrac_majma, medfrac_minma  = [], [], [], []
    quartfrac_majdw, quartfrac_mindw, quartfrac_majma, quartfrac_minma = [], [], [], []
    medfrac_majdiff, medfrac_mindiff, quartfrac_majdiff, quartfrac_mindiff  = [], [], [], []

    for snap in snapshots:  
        frac_majdw, frac_mindw, frac_majma, frac_minma = [], [], [], []
        frac_majdiff, frac_mindiff = [], []
        
        try:
            pair_path = f"TNG_{snap}_{reals}.hdf5"
            pair_data = h5py.File(f"{paths.path_pairs}{pair_path}", "r")
            
            if np.size(pair_data) == 0:
                raise EmptyFile
                
            redshift = pair_data['Header'].attrs['Redshift']
            
            if redshiftcutoff & ( redshift > redcutoff) :
                raise SkipRedshift
                
            if (len(pair_data['pairs']["hydro"]['Group ID']) == 0):    
                raise EmptyFile
            
            unpair = pair_data["unpaired"]["hydro"]
            unpairStells = np.array(unpair["Sub1 Stellar Mass"])
            unpairGroups = np.array(unpair["Group Mass"])
            unpairReals = np.array(unpair['Realization'])
            
            pair = pair_data["pairs"]["hydro"]
            priStell = np.array(pair["Sub1 Stellar Mass"])
            secStell = np.array(pair["Sub2 Stellar Mass"])
            pairGroups = np.array(pair["Group Mass"])
            pairReals = np.array(pair["Realization"])
            seps = np.array(pair["Separation"]) 
            
            # subset masks for unpaired
            majors = (secStell/priStell > 1/4)
            minors = (secStell/priStell > 1/10) & (secStell/priStell < 1/4)
            allpairs = (majors + minors)
            pair_lowsep = (seps > 10)
            
            ## dwarfs
            unpair_pri_dwarf = get_primmask(unpairStells, "dwarf")
            unpair_group_dwarf = get_groupmask(unpairGroups, "dwarf")                
            
            pair_pri_dwarf = get_primmask(priStell, "dwarf")
            pair_group_dwarf = get_groupmask(pairGroups, "dwarf")
            
                # defining combined masks 
            unpair_mask_dwarf = unpair_pri_dwarf & unpair_group_dwarf
            primary_mask_dwarf = pair_pri_dwarf & pair_group_dwarf
            pair_mask_dwarf = pair_pri_dwarf & pair_group_dwarf & pair_lowsep & allpairs
            
            ## massive
            unpair_pri_massive = get_primmask(unpairStells, "massive")
            unpair_group_massive = get_groupmask(unpairGroups, "massive")                
            
            pair_pri_massive = get_primmask(priStell, "massive")
            pair_group_massive = get_groupmask(pairGroups, "massive")

                # defining combined masks 
            unpair_mask_massive = unpair_pri_massive & unpair_group_massive
            primary_mask_massive = pair_pri_massive & pair_group_massive
            pair_mask_massive = pair_pri_massive & pair_group_massive & pair_lowsep & allpairs
                                                          
            for real in np.unique(unpairReals):                  
                # make realization masks
                unpair_real = unpairReals == real
                pair_real = pairReals == real

                # make count values for single realization
                numone_dwarf = np.count_nonzero(unpair_mask_dwarf & unpair_real)
                numtwo_dwarf = np.count_nonzero(primary_mask_dwarf & pair_real)
                numtot_dwarf = numone_dwarf + numtwo_dwarf
                nummaj_dwarf = np.count_nonzero(pair_mask_dwarf & pair_real & majors)
                nummin_dwarf = np.count_nonzero(pair_mask_dwarf & pair_real & minors)
                numpair_dwarf = np.count_nonzero(pair_mask_dwarf & pair_real)
                
                # make count values for single realization
                numone_massive = np.count_nonzero(unpair_mask_massive & unpair_real)
                numtwo_massive = np.count_nonzero(primary_mask_massive & pair_real)
                numtot_massive = numone_massive + numtwo_massive
                nummaj_massive = np.count_nonzero(pair_mask_massive & pair_real & majors)
                nummin_massive = np.count_nonzero(pair_mask_massive & pair_real & minors)
                numpair_massive = np.count_nonzero(pair_mask_massive & pair_real)
                
                if (numtot_dwarf == 0) or (numtot_massive == 0):
                    continue
                    
                # collect vals for all reals
                frac_majdw.append( nummaj_dwarf/numtot_dwarf ) 
                frac_mindw.append( nummin_dwarf/numtot_dwarf ) 
                frac_majma.append( nummaj_massive/numtot_massive ) 
                frac_minma.append( nummin_massive/numtot_massive ) 
                frac_majdiff.append( (nummaj_massive/numtot_massive) - (nummaj_dwarf/numtot_dwarf) ) 
                frac_mindiff.append( (nummin_massive/numtot_massive) - (nummaj_dwarf/numtot_dwarf) ) 
                    
            # create arrays of medians and quartiles~ 
            lower, upper = 0.5, 99.5                
            redshifts.append( redshift )
            
            medfrac_majdw.append( np.median( frac_majdw ) )
            medfrac_mindw.append( np.median( frac_mindw ) )
            medfrac_majma.append( np.median( frac_majma ) )
            medfrac_minma.append( np.median( frac_minma ) )
            medfrac_majdiff.append( np.median( frac_majdiff ) )
            medfrac_mindiff.append( np.median( frac_mindiff ) )
            
            quartfrac_majdw.append( np.percentile( frac_majdw, [lower,upper] ) )
            quartfrac_mindw.append( np.percentile( frac_mindw, [lower,upper] ) )
            quartfrac_majma.append( np.percentile( frac_majma, [lower,upper] ) )
            quartfrac_minma.append( np.percentile( frac_minma, [lower,upper] ) )
            quartfrac_majdiff.append( np.percentile( frac_majdiff, [lower,upper] ) )
            quartfrac_mindiff.append( np.percentile( frac_mindiff, [lower,upper] ) )

                                   
        except KeyError:
            if errorprint: print(f'skipping {snap} for KeyError. Please debug')
            continue
            
        except EmptyFile:
            if errorprint: print(f"skipping {snap}, empty file")
            continue
            
        except SkipRedshift:
            if errorprint: print(f"skipping {snap}, redshift out of range")
                
    count_dictionary = {
            "z": np.array(redshifts),
        
            "Median Major Dwarf": np.array(medfrac_majdw),
            "Median Minor Dwarf": np.array(medfrac_mindw),
            "Median Major Massive": np.array(medfrac_majma),
            "Median Minor Massive": np.array(medfrac_minma),
            "Median Major Difference": np.array(medfrac_majdiff),
            "Median Minor Difference": np.array(medfrac_mindiff),
        
            "Quartile Major Dwarf": np.array(quartfrac_majdw),
            "Quartile Minor Dwarf": np.array(quartfrac_mindw),
            "Quartile Major Massive": np.array(quartfrac_majma),
            "Quartile Minor Massive": np.array(quartfrac_minma),
            "Quartile Major Difference": np.array(quartfrac_majdiff),
            "Quartile Minor Difference": np.array(quartfrac_mindiff) }
    
    return count_dictionary


In [19]:
def make_pairfracdata(reals):
    #check if files exists
    filepath = f"{paths.path_plotdata}pairfrac.hdf5"
    if not os.path.isfile(filepath):
        print("file does not exist...")
        print("creating file")
        f = h5py.File(filepath, 'w')
        print("file created successfully. adding header...")
        header_dict = {"Simulation":"TNG100-1 (Hydro)",
                      "1000 Realization - quartile range":"0.5-99.5%"}


        dset = f.create_group('/Header')
        for key in header_dict.keys():
            dset.attrs[key] = header_dict[key]
            
        print("header added successfully")
    else:
        print("file exists...")
        f = h5py.File(filepath, 'r+')
        
        
    print("checking to see if data exists for this number of realizations")
    
    if f.get(f"{reals} Realizations") is not None:
        print("data already exists!")
        f.close()
              
    else:
        print("data does not exist...")
        print("creating data tables...")
        
        ratios = get_pairfrac(reals)

        for key, val in ratios.items():
            val = np.array(val)
            dset = f.create_dataset(f'/{reals} Realizations/{key}', 
                                    shape=val.shape,
                                    dtype=val.dtype)
            dset[:] = val
        print("data saved")        
        f.close()  

In [21]:
make_pairfracdata(10)
make_pairfracdata(100)
make_pairfracdata(1000)

file does not exist...
creating file
file created successfully. adding header...
header added successfully
checking to see if data exists for this number of realizations
data does not exist...
creating data tables...
data saved
file exists...
checking to see if data exists for this number of realizations
data does not exist...
creating data tables...
data saved
file exists...
checking to see if data exists for this number of realizations
data does not exist...
creating data tables...
data saved


# Sep and Vel evolution

In [28]:
def get_sepvel(reals, key, scaled=False, errorprint=False, redshiftcutoff=True): 
    snapshots = np.arange(0,100,1)
    snapshots = np.delete(snapshots, np.where(snapshots==48)[0])
    redcutoff = 4.2
        
    redshifts = []  
    med_majdw, med_mindw, med_majma, med_minma = [], [], [], []
    med_majdiff, med_mindiff, med_ddiff, med_mdiff = [], [], [], []
    quart_majdw, quart_mindw, quart_majma, quart_minma = [], [], [], []
    quart_majdiff, quart_mindiff, quart_ddiff, quart_mdiff  = [], [], [], []
            
    for snap in snapshots:  
        try:
            pair_path = f"TNG_{snap}_{reals}.hdf5"
            pair_data = h5py.File(f"{paths.path_pairs}{pair_path}", "r")
            
            if np.size(pair_data) == 0:
                raise EmptyFile
                
            redshift = pair_data['Header'].attrs['Redshift']
            
            if redshiftcutoff & ( redshift > redcutoff) :
                raise SkipRedshift
                
            if (len(pair_data['pairs']["hydro"]['Group ID']) == 0):    
                raise EmptyFile
                
            pair = pair_data["pairs"]["hydro"]
            priStell = np.array(pair["Sub1 Stellar Mass"])
            secStell = np.array(pair["Sub2 Stellar Mass"])
            pairGroups = np.array(pair["Group Mass"])
            pairGrRads = np.array(pair["Group Radius"])
            pairReals = np.array(pair["Realization"])
            seps = np.array(pair["Separation"]) 
            vels = np.array(pair["RelVel"])
                        
            majors = (secStell/priStell > 1/4)
            minors = (secStell/priStell > 1/10) & (secStell/priStell < 1/4)
            pair_lowsep = (seps > 10)
            
            if key == "Separation":
                scaleddat = seps / pairGrRads
            elif key == "RelVel":
                G = 4.3009173e4 # in km^2 kpc / (1e10M⊙ s^2)
                vvir = np.sqrt(G*pairGroups / pairGrRads)
                scaleddat = vels / (vvir)  
                
            if scaled: 
                dat = scaleddat
            elif key == "Separation":
                dat = seps
            elif key == "RelVel":
                dat = vels
            else:
                dat = np.array(pairs[key]) 
            
                 ## dwarfs
            pair_pri_dwarf = get_primmask(priStell, "dwarf")
            pair_group_dwarf = get_groupmask(pairGroups, "dwarf")
            
                # defining combined masks 
            pair_mask_dwarf = pair_pri_dwarf & pair_group_dwarf & pair_lowsep
            
            ## massive
            pair_pri_massive = get_primmask(priStell, "massive")
            pair_group_massive = get_groupmask(pairGroups, "massive")

                # defining combined masks 
            pair_mask_massive = pair_pri_massive & pair_group_massive & pair_lowsep
            
            real_majdw = []
            real_mindw = []
            real_majma = []
            real_minma = []
            real_majdiff = []
            real_mindiff = []
            real_ddiff = []
            real_mdiff = []
            
            realizations = np.unique( pairReals )

            for real in realizations:
                pair_real = pairReals == real

                mask_majdw = pair_real & pair_mask_dwarf & majors
                mask_mindw = pair_real & pair_mask_dwarf & minors
                mask_majma = pair_real & pair_mask_massive & majors
                mask_minma = pair_real & pair_mask_massive & minors

                majdw_xx = np.median( dat[mask_majdw] )
                mindw_xx = np.median( dat[mask_mindw] )
                majma_xx = np.median( dat[mask_majma] )
                minma_xx = np.median( dat[mask_minma] )

                real_majdw.append( majdw_xx )
                real_mindw.append( mindw_xx )
                real_majma.append( majma_xx )
                real_minma.append( minma_xx )
                real_majdiff.append( majma_xx - majdw_xx)
                real_mindiff.append( minma_xx - mindw_xx)
                real_ddiff.append( majdw_xx - mindw_xx)
                real_mdiff.append( majma_xx - minma_xx)

            lower, upper = 16,84         
            redshifts.append( redshift )
            
            med_majdw.append( np.median(real_majdw) )
            med_mindw.append( np.median(real_mindw) )
            med_majma.append( np.median(real_majma) )
            med_minma.append( np.median(real_minma) )
            med_majdiff.append( np.median(real_majdiff) )
            med_mindiff.append( np.median(real_mindiff) )
            med_ddiff.append( np.median(real_ddiff) )
            med_mdiff.append( np.median(real_mdiff) )
            
            quart_majdw.append( np.percentile( real_majdw, [lower,upper] ) )
            quart_mindw.append( np.percentile( real_mindw, [lower,upper] ) )
            quart_majma.append( np.percentile( real_majma, [lower,upper] ) )
            quart_minma.append( np.percentile( real_minma, [lower,upper] ) )
            quart_majdiff.append( np.percentile( real_majdiff, [lower,upper] ) )
            quart_mindiff.append( np.percentile( real_mindiff, [lower,upper] ) )
            quart_ddiff.append( np.percentile( real_ddiff, [lower,upper] ) )
            quart_mdiff.append( np.percentile( real_mdiff, [lower,upper] ) )

        except KeyError:
            if errorprint: print(f'skipping {snap} for KeyError. Please debug')
            continue
            
        except EmptyFile:
            if errorprint: print(f"skipping {snap}, empty file")
            continue
            
        except SkipRedshift:
            if errorprint: print(f"skipping {snap}, redshift out of range")
                
    scaled_dictionary = {"z": np.array(redshifts),

                        "Median Major Dwarf": np.array(med_majdw),
                        "Median Minor Dwarf": np.array(med_mindw),
                        "Median Major Massive": np.array(med_majma),
                        "Median Minor Massive": np.array(med_minma),
                        "Median Major Difference": np.array(med_majdiff),
                        "Median Minor Difference": np.array(med_mindiff),
                        "Median Dwarf Difference": np.array(med_ddiff),
                        "Median Massive Difference": np.array(med_mdiff),

                        "Quartile Major Dwarf": np.array(quart_majdw),
                        "Quartile Minor Dwarf": np.array(quart_mindw),
                        "Quartile Major Massive": np.array(quart_majma),
                        "Quartile Minor Massive": np.array(quart_minma),
                        "Quartile Major Difference": np.array(quart_majdiff),
                        "Quartile Minor Difference": np.array(quart_mindiff),
                        "Quartile Dwarf Difference": np.array(quart_ddiff),
                        "Quartile Massive Difference": np.array(quart_mdiff)}
    
    return scaled_dictionary


In [52]:
def make_sepveldata(reals):
        #check if files exists
    filepath = f"{paths.path_plotdata}sepvel.hdf5"
    if not os.path.isfile(filepath):
        print("file does not exist...")
        print("creating file")
        f = h5py.File(filepath, 'w')
        print("file created successfully. adding header...")
        header_dict = {"1000 Reals - Quartile Range":"16-84%",
                       "Simulation":"TNG100-1 (Hydro)"}

        dset = f.create_group('/Header')
        for key in header_dict.keys():
            dset.attrs[key] = header_dict[key]
            
        print("header added successfully")
    else:
        print("file exists...")
        f = h5py.File(filepath, 'r+')
        
        
    print("checking to see if data exists for this number of realizations")
    
    if f.get(f"{reals} Realizations") is not None:
        print("data already exists!")
        f.close()
              
    else:
        print("data does not exist...")
        print("creating data tables...")
        
        pairs = [("Separation",True),("Separation",False),("RelVel", True),("RelVel", False)]
        labels = ["Scaled Separation", "Separation", "Scaled Velocity", "Velocity"]         
        
        for pa, la in zip(pairs, labels):
            datums = get_sepvel(reals, pa[0], scaled=pa[1])
            print(f"collected data for {pa}")

            for key, val in datums.items():
                val = np.array(val)
                dset = f.create_dataset(f'/{reals} Realizations/{la}/{key}', 
                                        shape=val.shape,
                                        dtype=val.dtype)
                dset[:] = val
                
        f.close()
        print("data saved~")
    

In [30]:
testt = get_sepvel(10, "RelVel", scaled=True)

In [31]:
testt

{'z': array([4.17683491e+00, 4.00794511e+00, 3.70877426e+00, 3.49086137e+00,
        3.28303306e+00, 3.00813107e+00, 2.89578501e+00, 2.73314262e+00,
        2.57729027e+00, 2.44422570e+00, 2.31611074e+00, 2.20792547e+00,
        2.10326965e+00, 2.00202814e+00, 1.90408954e+00, 1.82268925e+00,
        1.74357057e+00, 1.66666956e+00, 1.60423452e+00, 1.53123903e+00,
        1.49551217e+00, 1.41409822e+00, 1.35757667e+00, 1.30237846e+00,
        1.24847261e+00, 1.20625808e+00, 1.15460271e+00, 1.11415056e+00,
        1.03551045e+00, 9.97294226e-01, 9.50531352e-01, 9.23000816e-01,
        8.86896938e-01, 8.51470901e-01, 8.16709979e-01, 7.91068249e-01,
        7.57441373e-01, 7.32636182e-01, 7.00106354e-01, 6.76110411e-01,
        6.44641841e-01, 6.21428745e-01, 5.98543288e-01, 5.75980845e-01,
        5.46392183e-01, 5.24565820e-01, 5.03047523e-01, 4.81832943e-01,
        4.60917794e-01, 4.40297849e-01, 4.19968942e-01, 3.99926965e-01,
        3.80167867e-01, 3.60687657e-01, 3.47853842e-01, 3.2

In [57]:
make_sepveldata(10)

file does not exist...
creating file
file created successfully. adding header...
header added successfully
checking to see if data exists for this number of realizations
data does not exist...
creating data tables...
collected data for ('Separation', True)
collected data for ('Separation', False)
collected data for ('RelVel', True)
collected data for ('RelVel', False)
data saved~


In [66]:
f = h5py.File(f"{paths.path_plotdata}sepvel.hdf5", 'r')

In [67]:
np.array(f['10 Realizations']["Separation"]["Median Dwarf Difference"])

array([ 4.15031387,  3.71732223,  5.71879622,  4.88309816,  6.28992762,
        5.8165457 ,  5.76829513,  5.31830854,  5.73811704,  6.75990854,
        6.34658907,  6.02112966,  7.15618795,  4.95292941,  5.55306841,
        6.88886272,  6.21601593,  5.92606282,  7.38699145,  8.14420185,
        8.44984178,  9.1258511 ,  8.2024838 ,  6.12870759,  6.59343332,
        7.2594458 ,  8.61704835,  4.23817197,  7.64768501,  8.9518017 ,
       13.28410105,  7.8299432 ,  9.217027  ,  6.26384857,  8.67960261,
        9.40885956, 10.24419211, 10.53082664,  9.63321497, 10.4976103 ,
       10.81600759, 10.41915926, 11.54586645, 14.13942388, 10.49593869,
       13.28054761, 12.96761123, 15.10727725, 13.20898145, 16.90326564,
        7.42979832, 10.35968554, 12.74512643, 14.90441467,  9.17277696,
       11.66414272, 12.72597282, 13.11785777, 10.45486323, 11.58677574,
       16.34838903, 12.86980491, 16.88520207, 14.2942911 , 17.93946417,
        7.49535382, 14.86295338, 14.97533294, 13.49400196, 23.94

In [68]:
f.close()

In [3]:
make_sepveldata(10)
make_sepveldata(100)
make_sepveldata(1000)

NameError: name 'paths' is not defined

In [80]:
f = h5py.File(f"{paths.path_plotdata}sepvel.hdf5",'r')

In [81]:
f.keys()

<KeysViewHDF5 ['10 Realizations', '100 Realizations', '1000 Realizations', 'Header']>

In [82]:
f["100 Realizations"]["Separation"].keys()

<KeysViewHDF5 ['Median Major Difference', 'Median Major Dwarf', 'Median Major Massive', 'Median Minor Difference', 'Median Minor Dwarf', 'Median Minor Massive', 'Quartile Major Difference', 'Quartile Major Dwarf', 'Quartile Major Massive', 'Quartile Minor Difference', 'Quartile Minor Dwarf', 'Quartile Minor Massive', 'z']>

In [65]:
f.close()

# Distributions

In [8]:
def get_sepveldist(reals, size, key, z, scaled=False):
    zloc = np.where( zs['z'] == z)[0]
    sim = "TNG"
    snapshot = zs['ztng'][zloc][0] 

    pair_path = f"{sim}_{snapshot}_{reals}.hdf5"
    pair_data = h5py.File(f"{paths.path_pairs}{pair_path}", "r")
    
    pairs = pair_data["pairs"]["hydro"]

    pri_stell = np.array(pairs["Sub1 Stellar Mass"])
    sec_stell = np.array(pairs["Sub2 Stellar Mass"])
    pairGroups = np.array(pairs["Group Mass"])
    pairGrRads = np.array(pairs["Group Radius"])
    seps = np.array(pairs["Separation"]) 
    vels = np.array(pairs["RelVel"]) 
    
    # masks            
    pair_pri = get_primmask(pri_stell, size)
    pair_group = get_groupmask(pairGroups, size)
    pair_sepcut = seps > 10
    
    pair_mask = pair_pri & pair_group & pair_sepcut
    
    majors = (sec_stell/pri_stell > 1/4)
    minors = (sec_stell/pri_stell > 1/10) & (sec_stell/pri_stell < 1/4)

    major_mask = pair_mask & majors
    minor_mask = pair_mask & minors
    
    if key == "Separation":
        scaleddat = seps / pairGrRads
    elif key == "RelVel":
        G = 4.3009173e4 # in km^2 kpc / (1e10M⊙ s^2)
        vvir = np.sqrt(G*pairGroups / pairGrRads)
        scaleddat = vels / (vvir)    
    
    if scaled: 
        dat = scaleddat
    elif key == "Separation":
        dat = seps
    elif key == "RelVel":
        dat = vels
    else:
        dat = np.array(pairs[key]) 

    majors = dat[major_mask]
    minors = dat[minor_mask]

    key_dict = {"major":majors, "minor":minors}
    

    return key_dict

In [13]:
def make_sepveldistdata(reals):
        #check if files exists
    filepath = f"{paths.path_plotdata}sepveldist.hdf5"
    if not os.path.isfile(filepath):
        print("file does not exist...")
        print("creating file")
        f = h5py.File(filepath, 'w')
        print("file created successfully. adding header...")
        header_dict = {"1000 Reals - Quartile Range":"16-84%",
                       "Simulation":"TNG100-1 (Hydro)"}

        dset = f.create_group('/Header')
        for key in header_dict.keys():
            dset.attrs[key] = header_dict[key]
            
        print("header added successfully")
    else:
        print("file exists...")
        f = h5py.File(filepath, 'r+')
        
        
    print("checking to see if data exists for this number of realizations")
    
    if f.get(f"{reals} Realizations") is not None:
        print("data already exists!")
        f.close()
              
    else:
        print("data does not exist...")
        print("creating data tables...")
        
        labels = ["Scaled Separation", "Separation", "Scaled Velocity", "Velocity"]         
        
        for z in zs["z"]:
            for size in ["dwarf","massive"]:
                ssep = get_sepveldist(reals, size, "Separation", z, scaled=True)
                sep = get_sepveldist(reals, size, "Separation", z, scaled=False)
                svel = get_sepveldist(reals, size, "RelVel", z, scaled=True)
                vel = get_sepveldist(reals, size, "RelVel", z, scaled=False)
              
                for datt, labb in zip([ssep,sep,svel,vel],labels):
                    for key, val in datt.items():
                        val = np.array(val)
                        dset = f.create_dataset(f'/{reals} Realizations/z={z}/{size}/{labb}/{key}', 
                                                shape=val.shape,
                                                dtype=val.dtype)
                        dset[:] = val
                    
            print(f"finished z={z}")        
                        
        f.close()
        print("data saved~")

In [15]:
make_sepveldistdata(10)
make_sepveldistdata(100)
make_sepveldistdata(1000)

file does not exist...
creating file
file created successfully. adding header...
header added successfully
checking to see if data exists for this number of realizations
data does not exist...
creating data tables...
finished z=0
finished z=1
finished z=2
finished z=3
finished z=4
data saved~
file exists...
checking to see if data exists for this number of realizations
data does not exist...
creating data tables...
finished z=0
finished z=1
finished z=2
finished z=3
finished z=4
data saved~
file exists...
checking to see if data exists for this number of realizations
data does not exist...
creating data tables...
finished z=0
finished z=1
finished z=2
finished z=3
finished z=4
data saved~


# Separation cuts

**by physical distance**

In [15]:
def get_pairfrac_sepcut(reals, sepcut, errorprint=False, redshiftcutoff=True):    
    snapshots = np.arange(0,100,1)
    snapshots = np.delete(snapshots, np.where(snapshots==48)[0])
    redcutoff = 4.2
        
    redshifts = []
    medfrac_majdw_cut, medfrac_mindw_cut, medfrac_majma_cut, medfrac_minma_cut  = [], [], [], []
    quartfrac_majdw_cut, quartfrac_mindw_cut, quartfrac_majma_cut, quartfrac_minma_cut = [], [], [], []
    medfrac_majdw_recovered, medfrac_mindw_recovered, medfrac_majma_recovered, medfrac_minma_recovered  = [], [], [], []
    quartfrac_majdw_recovered, quartfrac_mindw_recovered, quartfrac_majma_recovered, quartfrac_minma_recovered = [], [], [], []

    for snap in snapshots:  
        frac_majdw_cut, frac_mindw_cut, frac_majma_cut, frac_minma_cut = [], [], [], []
        frac_majdw_recovered, frac_mindw_recovered, frac_majma_recovered, frac_minma_recovered = [], [], [], []
        
        try:
            pair_path = f"TNG_{snap}_{reals}.hdf5"
            pair_data = h5py.File(f"{paths.path_pairs}{pair_path}", "r")
            
            if np.size(pair_data) == 0:
                raise EmptyFile
                
            redshift = pair_data['Header'].attrs['Redshift']
            
            if redshiftcutoff & ( redshift > redcutoff) :
                raise SkipRedshift
                
            if (len(pair_data['pairs']["hydro"]['Group ID']) == 0):    
                raise EmptyFile
            
            # unpaired masks
            unpair = pair_data["unpaired"]["hydro"]
            unpairStells = np.array(unpair["Sub1 Stellar Mass"])
            unpairGroups = np.array(unpair["Group Mass"])
            unpairReals = np.array(unpair['Realization'])
            
                ## unpaired dwarf primaries and groups
            unpair_pri_dwarf = get_primmask(unpairStells, "dwarf")
            unpair_group_dwarf = get_groupmask(unpairGroups, "dwarf")      
            
                ## unpaired massive primaries and groups
            unpair_pri_massive = get_primmask(unpairStells, "massive")
            unpair_group_massive = get_groupmask(unpairGroups, "massive")   
            
            # paired masks
            pair = pair_data["pairs"]["hydro"]
            priStell = np.array(pair["Sub1 Stellar Mass"])
            secStell = np.array(pair["Sub2 Stellar Mass"])
            pairGroups = np.array(pair["Group Mass"])
            pairReals = np.array(pair["Realization"])
            
                ## paired dwarf primaries and groups
            pair_pri_dwarf = get_primmask(priStell, "dwarf")
            pair_group_dwarf = get_groupmask(pairGroups, "dwarf")
            
                ## paired massive primaries and groups
            pair_pri_massive = get_primmask(priStell, "massive")
            pair_group_massive = get_groupmask(pairGroups, "massive")
            
            # major/minor pair masks
            majors = (secStell/priStell > 1/4)
            minors = (secStell/priStell > 1/10) & (secStell/priStell < 1/4)
            allpairs = (majors + minors)
            
            # sep pair mask
            seps = np.array(pair["Separation"]) 
            pair_lowsep = (seps > 10)
            pair_highsep = (seps < sepcut)
            
                ## dwarf primaries and pairs ~ 
            unpair_mask_dwarf = unpair_pri_dwarf & unpair_group_dwarf
            primary_mask_dwarf = pair_pri_dwarf & pair_group_dwarf            
            pair_mask_dwarf = pair_pri_dwarf & pair_group_dwarf & pair_lowsep & allpairs
            pair_mask_dwarf_cut = pair_mask_dwarf & pair_highsep
            
                # massive primaries and pairs
            unpair_mask_massive = unpair_pri_massive & unpair_group_massive
            primary_mask_massive = pair_pri_massive & pair_group_massive
            pair_mask_massive = pair_pri_massive & pair_group_massive & pair_lowsep  & allpairs
            pair_mask_massive_cut = pair_mask_massive & pair_highsep
                                                          
            for real in np.unique(unpairReals):                  
                # make realization masks
                unpair_real = unpairReals == real
                pair_real = pairReals == real

                # make count values for single realization
                numone_dwarf = np.count_nonzero(unpair_mask_dwarf & unpair_real)
                numtwo_dwarf = np.count_nonzero(primary_mask_dwarf & pair_real)
                numtot_dwarf = numone_dwarf + numtwo_dwarf
                nummaj_dwarf = np.count_nonzero(pair_mask_dwarf & pair_real & majors)
                nummin_dwarf = np.count_nonzero(pair_mask_dwarf & pair_real & minors)
                numpair_dwarf = np.count_nonzero(pair_mask_dwarf & pair_real)
                    # cut numbers
                nummaj_dwarf_cut = np.count_nonzero(pair_mask_dwarf_cut & pair_real & majors)
                nummin_dwarf_cut = np.count_nonzero(pair_mask_dwarf_cut & pair_real & minors)
                numpair_dwarf_cut = np.count_nonzero(pair_mask_dwarf_cut & pair_real)

                
                
                # make count values for single realization
                numone_massive = np.count_nonzero(unpair_mask_massive & unpair_real)
                numtwo_massive = np.count_nonzero(primary_mask_massive & pair_real)
                numtot_massive = numone_massive + numtwo_massive
                nummaj_massive = np.count_nonzero(pair_mask_massive & pair_real & majors)
                nummin_massive = np.count_nonzero(pair_mask_massive & pair_real & minors)
                numpair_massive = np.count_nonzero(pair_mask_massive & pair_real)
                    # cut numbers
                nummaj_massive_cut = np.count_nonzero(pair_mask_massive_cut & pair_real & majors)
                nummin_massive_cut = np.count_nonzero(pair_mask_massive_cut & pair_real & minors)
                numpair_massive_cut = np.count_nonzero(pair_mask_massive_cut & pair_real)
                
                if (numtot_dwarf == 0) or (numtot_massive == 0):
                    continue
                    
                # collect vals for all reals
                frac_majdw_cut.append( nummaj_dwarf_cut/numtot_dwarf ) 
                frac_mindw_cut.append( nummin_dwarf_cut/numtot_dwarf ) 
                frac_majma_cut.append( nummaj_massive_cut/numtot_massive ) 
                frac_minma_cut.append( nummin_massive_cut/numtot_massive ) 
                
                frac_majdw_recovered.append( nummaj_dwarf_cut/nummaj_dwarf )
                frac_mindw_recovered.append( nummin_dwarf_cut/nummin_dwarf ) 
                frac_majma_recovered.append( nummaj_massive_cut/nummaj_massive ) 
                frac_minma_recovered.append( nummin_massive_cut/nummin_massive ) 
                
                                    
            # create arrays of medians and quartiles~ 
            lower, upper = 0.5, 99.5                
            redshifts.append( redshift )
            
            medfrac_majdw_cut.append( np.median( frac_majdw_cut ) )
            medfrac_mindw_cut.append( np.median( frac_mindw_cut ) )
            medfrac_majma_cut.append( np.median( frac_majma_cut ) )
            medfrac_minma_cut.append( np.median( frac_minma_cut ) )
            medfrac_majdw_recovered.append( np.median( frac_majdw_recovered ) )
            medfrac_mindw_recovered.append( np.median( frac_mindw_recovered ) )
            medfrac_majma_recovered.append( np.median( frac_majma_recovered ) )
            medfrac_minma_recovered.append( np.median( frac_minma_recovered ) )
            
            quartfrac_majdw_cut.append( np.percentile( frac_majdw_cut, [lower,upper] ) )
            quartfrac_mindw_cut.append( np.percentile( frac_mindw_cut, [lower,upper] ) )
            quartfrac_majma_cut.append( np.percentile( frac_majma_cut, [lower,upper] ) )
            quartfrac_minma_cut.append( np.percentile( frac_minma_cut, [lower,upper] ) )
            quartfrac_majdw_recovered.append( np.percentile( frac_majdw_recovered, [lower,upper] ) )
            quartfrac_mindw_recovered.append( np.percentile( frac_mindw_recovered, [lower,upper] ) )
            quartfrac_majma_recovered.append( np.percentile( frac_majma_recovered, [lower,upper] ) )
            quartfrac_minma_recovered.append( np.percentile( frac_minma_recovered, [lower,upper] ) )
                                   
        except KeyError:
            if errorprint: print(f'skipping {snap} for KeyError. Please debug')
            continue
            
        except EmptyFile:
            if errorprint: print(f"skipping {snap}, empty file")
            continue
            
        except SkipRedshift:
            if errorprint: print(f"skipping {snap}, redshift out of range")
                
    count_dictionary = {
            "z": np.array(redshifts),
        
            "Median Major Dwarf": np.array(medfrac_majdw_cut),
            "Median Minor Dwarf": np.array(medfrac_mindw_cut),
            "Median Major Massive": np.array(medfrac_majma_cut),
            "Median Minor Massive": np.array(medfrac_minma_cut),
            "Frac Major Dwarf Recovered": np.array(medfrac_majdw_recovered),
            "Frac Minor Dwarf Recovered": np.array(medfrac_mindw_recovered),
            "Frac Major Massive Recovered": np.array(medfrac_majma_recovered),
            "Frac Minor Massive Recovered": np.array(medfrac_minma_recovered),
        
            "Quartile Major Dwarf": np.array(quartfrac_majdw_cut),
            "Quartile Minor Dwarf": np.array(quartfrac_mindw_cut),
            "Quartile Major Massive": np.array(quartfrac_majma_cut),
            "Quartile Minor Massive": np.array(quartfrac_minma_cut),
            "Quartile Major Dwarf Recovered": np.array(quartfrac_majdw_recovered),
            "Quartile Minor Dwarf Recovered": np.array(quartfrac_mindw_recovered),
            "Quartile Major Massive Recovered": np.array(quartfrac_majma_recovered),
            "Quartile Minor Massive Recovered": np.array(quartfrac_minma_recovered)
            }
    
    return count_dictionary


In [16]:
def make_pairfracdata_sepcut(reals, sepcut):
    #check if files exists
    filepath = f"{paths.path_plotdata}pairfrac_sepcut.hdf5"
    if not os.path.isfile(filepath):
        print("file does not exist...")
        print("creating file")
        f = h5py.File(filepath, 'w')
        print("file created successfully. adding header...")
        header_dict = {"Simulation":"TNG100-1 (Hydro)",
                      "1000 Realization - quartile range":"0.5-99.5%",
                      "Separation Cut":f"{sepcut}kpc"}


        dset = f.create_group('/Header')
        for key in header_dict.keys():
            dset.attrs[key] = header_dict[key]
            
        print("header added successfully")
    else:
        print("file exists...")
        f = h5py.File(filepath, 'r+')
        
        
    print("checking to see if data exists for this number of realizations")
    
    if f.get(f"{reals} Realizations/Sepcut {sepcut}kpc") is not None:
        print("data already exists!")
        f.close()
              
    else:
        print("data does not exist...")
        print("creating data tables...")
        
        ratios = get_pairfrac_sepcut(reals,sepcut)

        for key, val in ratios.items():
            val = np.array(val)
            dset = f.create_dataset(f'/{reals} Realizations/Sepcut {sepcut}kpc/{key}', 
                                    shape=val.shape,
                                    dtype=val.dtype)
            dset[:] = val
        print("data saved")        
        f.close()  

In [17]:

for sepcut in [50, 70, 100, 150, 200, 300]:
#     print(f"sepcut is: {sepcut}")
#     print("**** starting 10 ****")
#     make_pairfracdata_sepcut(10,sepcut)
    
#     print("**** starting 100 ****")
#     make_pairfracdata_sepcut(100,sepcut)    
    
    print("**** starting 1000 ****")
    make_pairfracdata_sepcut(1000,sepcut)    
    
# make_pairfracdata_sepcut(10,50) #
# make_pairfracdata_sepcut(10,70) # Snyder 2023
# make_pairfracdata_sepcut(10,100) #
# make_pairfracdata_sepcut(10,150) # Besla 2018


sepcut is: 50
**** starting 10 ****
file exists...
checking to see if data exists for this number of realizations
data already exists!
**** starting 100 ****
file exists...
checking to see if data exists for this number of realizations
data already exists!
**** starting 1000 ****
file exists...
checking to see if data exists for this number of realizations
data already exists!
sepcut is: 70
**** starting 10 ****
file exists...
checking to see if data exists for this number of realizations
data already exists!
**** starting 100 ****
file exists...
checking to see if data exists for this number of realizations
data already exists!
**** starting 1000 ****
file exists...
checking to see if data exists for this number of realizations
data already exists!
sepcut is: 100
**** starting 10 ****
file exists...
checking to see if data exists for this number of realizations
data already exists!
**** starting 100 ****
file exists...
checking to see if data exists for this number of realizations
dat

**by virial radius**

In [24]:
def get_pairfrac_vircut(reals, vircut, errorprint=False, redshiftcutoff=True):    
    snapshots = np.arange(0,100,1)
    snapshots = np.delete(snapshots, np.where(snapshots==48)[0])
    redcutoff = 4.2
        
    redshifts = []
    medfrac_majdw_cut, medfrac_mindw_cut, medfrac_majma_cut, medfrac_minma_cut  = [], [], [], []
    quartfrac_majdw_cut, quartfrac_mindw_cut, quartfrac_majma_cut, quartfrac_minma_cut = [], [], [], []
    medfrac_majdw_recovered, medfrac_mindw_recovered, medfrac_majma_recovered, medfrac_minma_recovered  = [], [], [], []
    quartfrac_majdw_recovered, quartfrac_mindw_recovered, quartfrac_majma_recovered, quartfrac_minma_recovered = [], [], [], []

    for snap in snapshots:  
        frac_majdw_cut, frac_mindw_cut, frac_majma_cut, frac_minma_cut = [], [], [], []
        frac_majdw_recovered, frac_mindw_recovered, frac_majma_recovered, frac_minma_recovered = [], [], [], []
        
        try:
            pair_path = f"TNG_{snap}_{reals}.hdf5"
            pair_data = h5py.File(f"{paths.path_pairs}{pair_path}", "r")
            
            if np.size(pair_data) == 0:
                raise EmptyFile
                
            redshift = pair_data['Header'].attrs['Redshift']
            
            if redshiftcutoff & ( redshift > redcutoff) :
                raise SkipRedshift
                
            if (len(pair_data['pairs']["hydro"]['Group ID']) == 0):    
                raise EmptyFile
            
            # unpaired masks
            unpair = pair_data["unpaired"]["hydro"]
            unpairStells = np.array(unpair["Sub1 Stellar Mass"])
            unpairGroups = np.array(unpair["Group Mass"])
            unpairReals = np.array(unpair['Realization'])
            
                ## unpaired dwarf primaries and groups
            unpair_pri_dwarf = get_primmask(unpairStells, "dwarf")
            unpair_group_dwarf = get_groupmask(unpairGroups, "dwarf")      
            
                ## unpaired massive primaries and groups
            unpair_pri_massive = get_primmask(unpairStells, "massive")
            unpair_group_massive = get_groupmask(unpairGroups, "massive")   
            
            # paired masks
            pair = pair_data["pairs"]["hydro"]
            priStell = np.array(pair["Sub1 Stellar Mass"])
            secStell = np.array(pair["Sub2 Stellar Mass"])
            pairGroups = np.array(pair["Group Mass"])
            pairRads = np.array(pair["Group Radius"])
            pairReals = np.array(pair["Realization"])
            
                ## paired dwarf primaries and groups
            pair_pri_dwarf = get_primmask(priStell, "dwarf")
            pair_group_dwarf = get_groupmask(pairGroups, "dwarf")
            
                ## paired massive primaries and groups
            pair_pri_massive = get_primmask(priStell, "massive")
            pair_group_massive = get_groupmask(pairGroups, "massive")
            
            # major/minor pair masks
            majors = (secStell/priStell > 1/4)
            minors = (secStell/priStell > 1/10) & (secStell/priStell < 1/4)
            allpairs = (majors + minors)
            
            # sep pair mask
            seps = np.array(pair["Separation"]) 
            pair_lowsep = (seps > 10)
            pair_highsep = (seps < vircut*pairRads)
            
                ## dwarf primaries and pairs ~ 
            unpair_mask_dwarf = unpair_pri_dwarf & unpair_group_dwarf
            primary_mask_dwarf = pair_pri_dwarf & pair_group_dwarf            
            pair_mask_dwarf = pair_pri_dwarf & pair_group_dwarf & pair_lowsep & allpairs
            pair_mask_dwarf_cut = pair_mask_dwarf & pair_highsep
            
                # massive primaries and pairs
            unpair_mask_massive = unpair_pri_massive & unpair_group_massive
            primary_mask_massive = pair_pri_massive & pair_group_massive
            pair_mask_massive = pair_pri_massive & pair_group_massive & pair_lowsep  & allpairs
            pair_mask_massive_cut = pair_mask_massive & pair_highsep
                                                          
            for real in np.unique(unpairReals):                  
                # make realization masks
                unpair_real = unpairReals == real
                pair_real = pairReals == real

                # make count values for single realization
                numone_dwarf = np.count_nonzero(unpair_mask_dwarf & unpair_real)
                numtwo_dwarf = np.count_nonzero(primary_mask_dwarf & pair_real)
                numtot_dwarf = numone_dwarf + numtwo_dwarf
                nummaj_dwarf = np.count_nonzero(pair_mask_dwarf & pair_real & majors)
                nummin_dwarf = np.count_nonzero(pair_mask_dwarf & pair_real & minors)
                numpair_dwarf = np.count_nonzero(pair_mask_dwarf & pair_real)
                    # cut numbers
                nummaj_dwarf_cut = np.count_nonzero(pair_mask_dwarf_cut & pair_real & majors)
                nummin_dwarf_cut = np.count_nonzero(pair_mask_dwarf_cut & pair_real & minors)
                numpair_dwarf_cut = np.count_nonzero(pair_mask_dwarf_cut & pair_real)

                
                
                # make count values for single realization
                numone_massive = np.count_nonzero(unpair_mask_massive & unpair_real)
                numtwo_massive = np.count_nonzero(primary_mask_massive & pair_real)
                numtot_massive = numone_massive + numtwo_massive
                nummaj_massive = np.count_nonzero(pair_mask_massive & pair_real & majors)
                nummin_massive = np.count_nonzero(pair_mask_massive & pair_real & minors)
                numpair_massive = np.count_nonzero(pair_mask_massive & pair_real)
                    # cut numbers
                nummaj_massive_cut = np.count_nonzero(pair_mask_massive_cut & pair_real & majors)
                nummin_massive_cut = np.count_nonzero(pair_mask_massive_cut & pair_real & minors)
                numpair_massive_cut = np.count_nonzero(pair_mask_massive_cut & pair_real)
                
                if (numtot_dwarf == 0) or (numtot_massive == 0):
                    continue
                    
                # collect vals for all reals
                frac_majdw_cut.append( nummaj_dwarf_cut/numtot_dwarf ) 
                frac_mindw_cut.append( nummin_dwarf_cut/numtot_dwarf ) 
                frac_majma_cut.append( nummaj_massive_cut/numtot_massive ) 
                frac_minma_cut.append( nummin_massive_cut/numtot_massive ) 
                
                frac_majdw_recovered.append( nummaj_dwarf_cut/nummaj_dwarf )
                frac_mindw_recovered.append( nummin_dwarf_cut/nummin_dwarf ) 
                frac_majma_recovered.append( nummaj_massive_cut/nummaj_massive ) 
                frac_minma_recovered.append( nummin_massive_cut/nummin_massive ) 
                
                                    
            # create arrays of medians and quartiles~ 
            lower, upper = 0.5, 99.5                
            redshifts.append( redshift )
            
            medfrac_majdw_cut.append( np.median( frac_majdw_cut ) )
            medfrac_mindw_cut.append( np.median( frac_mindw_cut ) )
            medfrac_majma_cut.append( np.median( frac_majma_cut ) )
            medfrac_minma_cut.append( np.median( frac_minma_cut ) )
            medfrac_majdw_recovered.append( np.median( frac_majdw_recovered ) )
            medfrac_mindw_recovered.append( np.median( frac_mindw_recovered ) )
            medfrac_majma_recovered.append( np.median( frac_majma_recovered ) )
            medfrac_minma_recovered.append( np.median( frac_minma_recovered ) )
            
            quartfrac_majdw_cut.append( np.percentile( frac_majdw_cut, [lower,upper] ) )
            quartfrac_mindw_cut.append( np.percentile( frac_mindw_cut, [lower,upper] ) )
            quartfrac_majma_cut.append( np.percentile( frac_majma_cut, [lower,upper] ) )
            quartfrac_minma_cut.append( np.percentile( frac_minma_cut, [lower,upper] ) )
            quartfrac_majdw_recovered.append( np.percentile( frac_majdw_recovered, [lower,upper] ) )
            quartfrac_mindw_recovered.append( np.percentile( frac_mindw_recovered, [lower,upper] ) )
            quartfrac_majma_recovered.append( np.percentile( frac_majma_recovered, [lower,upper] ) )
            quartfrac_minma_recovered.append( np.percentile( frac_minma_recovered, [lower,upper] ) )
                                   
        except KeyError:
            if errorprint: print(f'skipping {snap} for KeyError. Please debug')
            continue
            
        except EmptyFile:
            if errorprint: print(f"skipping {snap}, empty file")
            continue
            
        except SkipRedshift:
            if errorprint: print(f"skipping {snap}, redshift out of range")
                
    count_dictionary = {
            "z": np.array(redshifts),
        
            "Median Major Dwarf": np.array(medfrac_majdw_cut),
            "Median Minor Dwarf": np.array(medfrac_mindw_cut),
            "Median Major Massive": np.array(medfrac_majma_cut),
            "Median Minor Massive": np.array(medfrac_minma_cut),
            "Frac Major Dwarf Recovered": np.array(medfrac_majdw_recovered),
            "Frac Minor Dwarf Recovered": np.array(medfrac_mindw_recovered),
            "Frac Major Massive Recovered": np.array(medfrac_majma_recovered),
            "Frac Minor Massive Recovered": np.array(medfrac_minma_recovered),
        
            "Quartile Major Dwarf": np.array(quartfrac_majdw_cut),
            "Quartile Minor Dwarf": np.array(quartfrac_mindw_cut),
            "Quartile Major Massive": np.array(quartfrac_majma_cut),
            "Quartile Minor Massive": np.array(quartfrac_minma_cut),
            "Quartile Major Dwarf Recovered": np.array(quartfrac_majdw_recovered),
            "Quartile Minor Dwarf Recovered": np.array(quartfrac_mindw_recovered),
            "Quartile Major Massive Recovered": np.array(quartfrac_majma_recovered),
            "Quartile Minor Massive Recovered": np.array(quartfrac_minma_recovered)
            }
    
    return count_dictionary


In [31]:
def make_pairfracdata_vircut(reals, vircut):
    #check if files exists
    filepath = f"{paths.path_plotdata}pairfrac_vircut_full.hdf5"
    if not os.path.isfile(filepath):
        print("file does not exist...")
        print("creating file")
        f = h5py.File(filepath, 'w')
        print("file created successfully. adding header...")
        header_dict = {"Simulation":"TNG100-1 (Hydro)",
                      "1000 Realization - quartile range":"0.5-99.5%"}


        dset = f.create_group('/Header')
        for key in header_dict.keys():
            dset.attrs[key] = header_dict[key]
            
        print("header added successfully")
    else:
        print("file exists...")
        f = h5py.File(filepath, 'r+')
        
        
    print("checking to see if data exists for this number of realizations")
    
    if f.get(f"{reals} Realizations/Vircut {vircut}") is not None:
        print("data already exists!")
        f.close()
              
    else:
        print("data does not exist...")
        print("creating data tables...")
        
        ratios = get_pairfrac_vircut(reals,vircut)

        for key, val in ratios.items():
            val = np.array(val)
            dset = f.create_dataset(f'/{reals} Realizations/Vircut {vircut}/{key}', 
                                    shape=val.shape,
                                    dtype=val.dtype)
            dset[:] = val
        print("data saved")        
        f.close()  

In [33]:
vircuts = [0.25,0.5,1,1.5,2,2.5]
reals = [10, 100, 1000]

for real in reals:
    print(f"**** starting {real} realizations ****")
    for vircut in vircuts:
        print(f"virial is: {vircut}")

        make_pairfracdata_vircut(real,vircut)
    

**** starting 10 realizations ****
virial is: 0.25
file exists...
checking to see if data exists for this number of realizations
data already exists!
virial is: 0.5
file exists...
checking to see if data exists for this number of realizations
data already exists!
virial is: 1
file exists...
checking to see if data exists for this number of realizations
data already exists!
virial is: 1.5
file exists...
checking to see if data exists for this number of realizations
data already exists!
virial is: 2
file exists...
checking to see if data exists for this number of realizations
data already exists!
virial is: 2.5
file exists...
checking to see if data exists for this number of realizations
data already exists!
**** starting 100 realizations ****
virial is: 0.25
file exists...
checking to see if data exists for this number of realizations
data does not exist...
creating data tables...
data saved
virial is: 0.5
file exists...
checking to see if data exists for this number of realizations
dat

In [ ]:
a
for vircut in [0.5,1,1.5,2,2.5]:
    print(f"sepcut is: {vircut}")
    print("**** starting 10 ****")
    make_pairfracdata_sepcut(10,vircut)
    
    print("**** starting 100 ****")
    make_pairfracdata_sepcut(100,vircut)    
    
    print("**** starting 1000 ****")
    make_pairfracdata_sepcut(1000,vircut)    
    
# make_pairfracdata_sepcut(10,50) #
# make_pairfracdata_sepcut(10,70) # Snyder 2023
# make_pairfracdata_sepcut(10,100) #
# make_pairfracdata_sepcut(10,150) # Besla 2018


In [13]:
tester = get_pairfrac_sepcut(10, 50, errorprint=False, redshiftcutoff=True)

In [14]:
tester.keys()

dict_keys(['z', 'Median Major Dwarf', 'Median Minor Dwarf', 'Median Major Massive', 'Median Minor Massive', 'Frac Major Dwarf Recovered', 'Frac Minor Dwarf Recovered', 'Frac Major Massive Recovered', 'Frac Minor Massive Recovered', 'Quartile Major Dwarf', 'Quartile Minor Dwarf', 'Quartile Major Massive', 'Quartile Minor Massive', 'Quartile Major Dwarf Recovered'])

In [17]:
tester['Median Major Dwarf'][0:10]

array([0.07847577, 0.07887069, 0.07339582, 0.07194664, 0.06953869,
       0.06611652, 0.06393404, 0.06407182, 0.06183954, 0.059131  ])

In [21]:
tester["Frac Major Dwarf Recovered"]

array([0.60982658, 0.60493624, 0.57076118, 0.56065726, 0.54203146,
       0.53058157, 0.50883725, 0.51007527, 0.50957423, 0.48280846,
       0.47446969, 0.47764149, 0.47054536, 0.47523125, 0.47021844,
       0.45184615, 0.44632103, 0.44472973, 0.43105776, 0.42673449,
       0.42294315, 0.40412224, 0.407767  , 0.39735095, 0.40086798,
       0.39142738, 0.38662259, 0.38982796, 0.36373714, 0.35531717,
       0.34891549, 0.35912805, 0.35882905, 0.3616776 , 0.34056667,
       0.3283059 , 0.33783288, 0.32227173, 0.33021008, 0.32031746,
       0.32287681, 0.31488475, 0.32011985, 0.30974076, 0.31443362,
       0.29750466, 0.29782689, 0.29577776, 0.28338898, 0.28054239,
       0.29858629, 0.29595993, 0.27391466, 0.26358615, 0.2747796 ,
       0.26798645, 0.26559285, 0.2548034 , 0.25210515, 0.26633128,
       0.25607339, 0.25052433, 0.26670385, 0.24497241, 0.2539489 ,
       0.25548204, 0.23159256, 0.23658268, 0.21869015, 0.22241378,
       0.2175508 , 0.21192839, 0.21182655, 0.20560499, 0.20520

# Comparison plot (lit review) 


In [1]:
redshift = [0.25, 0.75, 1.25, 
            1.75, 2.25, 3, 
            4, 5.5]

subfind_full = [0.28408361844787400, 0.3967792682835680, 0.3105562443495830, 
                0.2548518480347080, 0.43983420079419700, 0.34714255831491100, 
                0.3711320119287430, 0.21564351220864500]

subfind_detection = [0.16008590055114600, 0.3301736741546460, 0.26277792288463100, 
                     0.23118845501327600, 0.3413915132625050, 0.2499329172046900, 
                     0.310556244349583, 0.12883571609350400]

source_detection = [0.48756106896697400, 0.3732044346340160, 0.3376105125784730, 
                    0.28093731775220400, 0.41255098653991600, 0.310556244349583, 
                    0.35993769212646000, 0.17258426971826300]
